### Sentiment analysis ImDB

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

In [2]:
imdb,info=tfds.load('imdb_reviews',with_info=True,as_supervised=True)

In [3]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [25]:
train_data,test_data=imdb['train'],imdb['test']

In [27]:
train_sentence=[]
train_label=[]
for lines in train_data:
    train_sentence.append(str(lines[0].numpy()))
    train_label.append(lines[1].numpy())
    

In [28]:
test_sentence=[]
test_label=[]
for lines in test_data:
    test_sentence.append(str(lines[0].numpy()))
    test_label.append(lines[1].numpy())
    

In [29]:
train_sentence=np.array(train_sentence)
test_sentence=np.array(test_sentence)
train_label=np.array(tain_label,dtype=np.int16)
test_label=np.array(test_label,dtype=np.int16)

In [30]:
type(train_sentence[3])

numpy.str_

In [31]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [38]:
num_words=10000
embed_dim=16
padding='post'
oov_text='<OOV>'
max_length=120

In [39]:
token=Tokenizer(num_words=num_words,oov_token=oov_text)
token.fit_on_texts(train_sentence)
word_index=token.word_index

In [40]:
word_index['the']

2

In [41]:
vocab_size=len(word_index)
vocab_size

86539

In [42]:
#3Sequences
train_seq=token.texts_to_sequences(train_sentence)
test_seq=token.texts_to_sequences(test_sentence)

In [43]:
## Padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [44]:
train_pad_seq=pad_sequences(train_seq,maxlen=15,padding='pre')
test_pad_seq=pad_sequences(test_seq,maxlen=15,padding='pre')

In [45]:
train_pad_seq[2]

array([   5,  746,    1,   17,   12, 2807,  115,   42,    6, 4518,  131,
        424,    9, 6176,   47], dtype=int32)

In [46]:
## model creation
from tensorflow.keras.layers import Dense,Flatten,Embedding,LSTM

In [49]:
model=keras.Sequential()
model.add(Embedding(input_dim=num_words,output_dim=32,input_length=15,))
model.add(LSTM(20))
model.add(Flatten())
model.add(Dense(units=128,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 15, 32)            320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 20)                4240      
_________________________________________________________________
flatten_1 (Flatten)          (None, 20)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               2688      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 327,057
Trainable params: 327,057
Non-trainable params: 0
_________________________________________________________________


In [51]:
model.fit(train_pad_seq,train_label,validation_data=(test_pad_seq,test_label),epochs=10)

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 31s 1ms/sample - loss: 0.5473 - accuracy: 0.7056 - val_loss: 0.5048 - val_accuracy: 0.7454
Epoch 2/10
25000/25000 [==============================] - 20s 792us/sample - loss: 0.4301 - accuracy: 0.7943 - val_loss: 0.5111 - val_accuracy: 0.7442
Epoch 3/10
25000/25000 [==============================] - 20s 797us/sample - loss: 0.3657 - accuracy: 0.8244 - val_loss: 0.5803 - val_accuracy: 0.7410
Epoch 4/10
25000/25000 [==============================] - 20s 801us/sample - loss: 0.3059 - accuracy: 0.8547 - val_loss: 0.6246 - val_accuracy: 0.7332
Epoch 5/10
25000/25000 [==============================] - 19s 751us/sample - loss: 0.2480 - accuracy: 0.8848 - val_loss: 0.7518 - val_accuracy: 0.7214
Epoch 6/10
25000/25000 [==============================] - 20s 791us/sample - loss: 0.1976 - accuracy: 0.9102 - val_loss: 0.9239 - val_accuracy: 0.7176
Epoch 7/10
25000/25000 [======================

### WE will save the weights and text to visualize in ts projector

In [52]:
embed_layer=model.get_layer('embedding_1')

In [53]:
embed_layer

In [68]:
embed_layer.get_weights()[0]
arr_weights=embed_layer.get_weights()[0]

In [ ]:
for embed_layer.get_weights()[0][i]

In [60]:
### Reversse the word index
reverse_word_index=dict([(w,i) for i,w in word_index.items()])

In [61]:
reverse_word_index[234]

'performance'

In [62]:
 ### Create files
import io
word_file=io.open('word_file.tsv','w',encoding='utf-8')
vector_file=io.open('vector_file.tsv','w',encoding='utf-8')

In [74]:
for i in range(num_words):
    word_file.write(reverse_word_index[i+1])
    vector_file.write('\t'.join([str(x) for x in arr_weights[i]]) +'\n')